In [2]:
import requests
import json
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pandas_ta as ta
import pickle
import time
from pytrends.request import TrendReq
pytrends = TrendReq(hl='en-US', tz=360)
import os
apikey = 'CH59E4O2KQ0HB7AN'
#inpuut = input('Historyczne sprawdzenie wciśnij A, livedata B: ')
ts = TimeSeries(apikey, output_format='pandas')
def getData(stock):
    eod, meta = ts.get_daily(symbol=stock)
    return(eod)

def downloadTrends(keyword, first ,last):
    kw_list = [keyword]
    date1 = last+' '+first

    pytrends.build_payload(kw_list, cat=0, timeframe=date1, geo='', gprop='')
    time.sleep(6)
    data = pytrends.interest_over_time()
    data= data.drop(labels=['isPartial'],axis='columns')
    return data


listt = []

def makeDataFrame(date, stock):
    jsonn = getData(stock)
    #date = date + ' 00:00:00'
    jsonn.reset_index(inplace=True)
    jsonn['date'] = jsonn['date'].astype(str)
    datee = jsonn['date'].to_list()
    poss = datee.index(date)
    jsonn = jsonn.iloc[poss:]        
    jsonn = jsonn.set_axis(['date','Open','High','Low','Close','Volume'], axis=1)
    jsonn = jsonn[::-1]
    
    jsonn.ta.sma(length=50,append=True) 
    jsonn.ta.rsi(length=14,append=True) 
    
    jsonn = jsonn.tail(20)
    
    last_d = jsonn['date'].iloc[0]
    first_d = jsonn['date'].iloc[-1] 
    
    trends = downloadTrends(stock,first_d,last_d)
    
    jsonn['Adjusted_close'] = trends[stock]
    jsonn.reset_index(inplace=True)
    
    file_max = pd.read_csv('Trends5per/'+stock+'TRENDS.csv')
    company  = file_max[['Date','Open','High','Low','Close','Volume','Adjusted_close','ema','rsi']]
    company = company.rename(columns={'Date':'date','Volume':'volume','Open':'open','Close':'close','Low':'low','High':'high','ema':'SMA_50','rsi':'RSI_14'})
    
    for x in range(0,len(jsonn)):
        jsonn['Adjusted_close'][x] = trends[stock][x]
    df1 = jsonn[['date','open','high','low','close','volume','Adjusted_close','SMA_50','RSI_14']]
    for x in range(len(df1)):
        company = company.append(df1.iloc[x])
    df = company
    sc= MinMaxScaler(feature_range=(0,1))
    df['Open0']=sc.fit_transform(df['open'].to_numpy().reshape(-1, 1))
    df['High0']=sc.fit_transform(df['high'].to_numpy().reshape(-1, 1))
    df['Low0']=sc.fit_transform(df['low'].to_numpy().reshape(-1, 1))
    df['Close0']=sc.fit_transform(df['close'].to_numpy().reshape(-1, 1))
    df['Adjusted_close0']=sc.fit_transform(df['Adjusted_close'].to_numpy().reshape(-1, 1))
    df['Volume0']=sc.fit_transform(df['volume'].to_numpy().reshape(-1, 1))
    df['rsi0']=sc.fit_transform(df['RSI_14'].to_numpy().reshape(-1, 1))
    df['ema0']=sc.fit_transform(df['SMA_50'].to_numpy().reshape(-1, 1))

    
    for i in range(1,20):
        df['Open'+str(i)] = df['Open0'].shift(i)
        df['High'+str(i)] = df['High0'].shift(i)
        df['Low'+str(i)] = df['Low0'].shift(i)
        df['Close'+str(i)] = df['Close0'].shift(i)
        df['Adjusted_close'+str(i)] = df['Adjusted_close0'].shift(i)
        df['Volume'+str(i)] = df['Volume0'].shift(i)
        df['rsi'+str(i)] = df['rsi0'].shift(i)
        df['ema'+str(i)] = df['ema0'].shift(i)
    df.to_csv('TESTTT')
    return df
    
files = os.listdir('MachineLearning5per')     
#x = makeDataFrame('2021-04-22','IBM')
files.pop(0)
dictttt={}
for x in files:
    time.sleep(10)
    print('START ',x[:-10])
    loaded_model = pickle.load(open('MachineLearning5per/'+x, 'rb'))
    g = makeDataFrame('2021-04-27',x[:-10])
    df1 = g[['Open0', 'High0', 'Low0', 'Close0', 'Adjusted_close0', 'Volume0', 'rsi0', 'ema0', 'Open1', 'High1', 'Low1', 'Close1', 'Adjusted_close1', 'Volume1', 'rsi1', 'ema1', 'Open2', 'High2', 'Low2', 'Close2', 'Adjusted_close2', 'Volume2', 'rsi2', 'ema2', 'Open3', 'High3', 'Low3', 'Close3', 'Adjusted_close3', 'Volume3', 'rsi3', 'ema3', 'Open4', 'High4', 'Low4', 'Close4', 'Adjusted_close4', 'Volume4', 'rsi4', 'ema4', 'Open5', 'High5', 'Low5', 'Close5', 'Adjusted_close5', 'Volume5', 'rsi5', 'ema5', 'Open6', 'High6', 'Low6', 'Close6', 'Adjusted_close6', 'Volume6', 'rsi6', 'ema6', 'Open7', 'High7', 'Low7', 'Close7', 'Adjusted_close7', 'Volume7', 'rsi7', 'ema7', 'Open8', 'High8', 'Low8', 'Close8', 'Adjusted_close8', 'Volume8', 'rsi8', 'ema8', 'Open9', 'High9', 'Low9', 'Close9', 'Adjusted_close9', 'Volume9', 'rsi9', 'ema9', 'Open10', 'High10', 'Low10', 'Close10', 'Adjusted_close10', 'Volume10', 'rsi10', 'ema10', 'Open11', 'High11', 'Low11', 'Close11', 'Adjusted_close11', 'Volume11', 'rsi11', 'ema11', 'Open12', 'High12', 'Low12', 'Close12', 'Adjusted_close12', 'Volume12', 'rsi12', 'ema12', 'Open13', 'High13', 'Low13', 'Close13', 'Adjusted_close13', 'Volume13', 'rsi13', 'ema13', 'Open14', 'High14', 'Low14', 'Close14', 'Adjusted_close14', 'Volume14', 'rsi14', 'ema14', 'Open15', 'High15', 'Low15', 'Close15', 'Adjusted_close15', 'Volume15', 'rsi15', 'ema15', 'Open16', 'High16', 'Low16', 'Close16', 'Adjusted_close16', 'Volume16', 'rsi16', 'ema16', 'Open17', 'High17', 'Low17', 'Close17', 'Adjusted_close17', 'Volume17', 'rsi17', 'ema17', 'Open18', 'High18', 'Low18', 'Close18', 'Adjusted_close18', 'Volume18', 'rsi18', 'ema18', 'Open19', 'High19', 'Low19', 'Close19', 'Adjusted_close19', 'Volume19', 'rsi19', 'ema19']]
    
    predic = df1.iloc[[-1]].to_numpy()
    
    vb = loaded_model.predict(predic)
    if vb == 1:
        bv = loaded_model.predict_proba(predic)
        listt.append(bv[0][1])
        dictttt[x[:-10]] = bv[0][1]
        print('BINGOOOOOOOO')
    print(vb)

my_keys = sorted(dictttt, key=dictttt.get, reverse=True)[:4]
print(my_keys)
#x = getData('IBM')

 

START  ABT


<ipython-input-2-ee60bd9adc20>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jsonn['Adjusted_close'][x] = trends[stock][x]


[0.]
START  ACN


<ipython-input-2-ee60bd9adc20>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jsonn['Adjusted_close'][x] = trends[stock][x]


[0.]
START  ADBE


...

[0.]
START  ZBH


<ipython-input-2-ee60bd9adc20>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jsonn['Adjusted_close'][x] = trends[stock][x]


[0.]
START  ZBRA


<ipython-input-2-ee60bd9adc20>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jsonn['Adjusted_close'][x] = trends[stock][x]


[0.]
['APH', 'VMC', 'DRI', 'VIAC']


In [4]:
my_keys = sorted(dictttt, key=dictttt.get, reverse=True)[:4]
print(my_keys)

['MS', 'BKNG', 'APH', 'SHW']


In [93]:
print(listt)

[0.7623926959128003, 0.9998076956807298, 0.9333603593972876, 0.9759118769799688, 0.9689423530405328, 0.5707892084956205, 0.6936443776839292, 0.6692018779969959, 0.5039773652508166, 0.6037585671797605, 0.6589113625516141, 0.5150306473783809, 0.9987511320530585, 0.9990151812413381, 0.7545988940812578, 0.5172406893412911, 0.5800034934636106]
